# Aprendizaje por Refuerzo
## Sistema de Recomendación con Aprendizaje por Refuerzo Integrando Ratings

In [1]:
# pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: c:\Users\Ed\.pyenv\pyenv-win\versions\3.11.5\python.exe -m pip install --upgrade pip


In [ ]:
# pip install fastparquet

In [2]:
# librerías necesarias

import pandas as pd
import random
import numpy as np
import csv


# Datos de interacciones de usuarios con películas, 
# la best_movie_id es el id de la película vista en el mismo día con mejor calificación dada por el usuario
data = pd.read_parquet(r"Netflix_Prize_data\netflix_data_sample.parquet")
# vista previa de los datos
data.head()

,movie_id,customer_id,rating,best_movie_id,best_rating
0,14367,10,5,9340,5
1,571,10,4,9340,5
2,2122,10,4,9340,5
3,6972,10,4,9340,5
4,15124,10,4,9340,5


# Ambiente

In [3]:
class Environment:
    def __init__(self, data, movie_titles):
        # el ambiente tiene en memoria el dataframe con los datos de las interacciones de usuarios
        self.data = data

        # se escoge como punto de partida cualquier contenido movie_id
        self.initial_state = data.sample(1)["movie_id"].iloc[0]

        # se inicializa el estado actual con el estado inicial
        self.state = self.initial_state

        # strikes controla la terminación del episodio, 1 strikes y el episodio termina
        self.strikes = 0
        # recompensa por defecto si se llega a 1 strike, está fuera (out)
        self.reward_out = -10

        # se guardan los títulos de las películas para poder mostrarlas en el entorno
        self.movie_titles = movie_titles



    def reset_strikes(self):
        """
        Reinicia los strikes a 0
        """
        self.strikes = 0
       
    def is_terminal(self):
        """
        Determina si el episodio ha terminado, si el agente ha llegado a 1 strike
        """
        return self.strikes == 1

    def get_current_state(self):
        """
        Retorna el estado actual (el movie_id actual)

        Returns
        -------
        int
            Identificador de la película actual (movie_id)
        """
        return self.state

    def get_movie_name(self, movie_id):
        """
        Retorna el nombre de la película dado el movie_id

        Parameters
        ----------
        movie_id : int
            Identificador de la película

        Returns
        -------
        str
            Nombre de la película
        """
        return self.movie_titles[self.movie_titles.movie_id == movie_id].title.iloc[0]


    def get_possible_actions(self, state):
        """
        Retorna las acciones posibles dado un estado (movie_id), 
        las acciones serán cambiar a otro contenido dentro de los candidatos, los cuales son el grupo de películas con mejor calificación
        media del usuario que ha calificado la película actual
        """
        # obtener las películas candidatas, puntuación media dadas por los usuarios que vieron el contenido actual
        candidates = data[data.movie_id == state].groupby("best_movie_id")["best_rating"].mean().sort_values(ascending=False)
        # filtro para que solo se escojan las películas con mayor rating dentro del grupo de candidatos
        candidates = candidates[(candidates==candidates.max())].index

        return candidates
    
    def do_action(self, action):
        """
        Realiza una acción en el entorno, cambiar de contenido a recomendar

        Parameters
        ----------
        action : int
            Identificador de la película a recomendar (movie_id)

        Returns
        -------
        float
            Recompensa de realizar la acción
        int
            Nuevo estado (movie_id)
        bool
            True si el episodio ha terminado, False de lo contrario
        """
        
        reward = 0
        done = False

        if self.is_terminal():
            # si es terminal, se asigna la recompensa por defecto y se termina el episodio
            reward = self.reward_out
            done = True
        else:
            # se obtiene la calificación media de los usuarios que vieron el contenido a recomendar
            rating = self.data[self.data.movie_id == action].rating.mean()
            # se escala la recompensa en función de la calificación media
            reward = self.reward_scalation(rating)

            # si el contenido recomendado tiene una calificación media menor a 3, se considera un strike para el agente,
            # de lo contrario se reinician los strikes
            if reward<0:
                self.strikes += 1
            else:
                self.reset_strikes()

        # recordar que la acción es el cambio de contenido, el nuevo estado es el nuevo contenido (id_movie a recomendar)
        self.state = action
        return reward, self.state, done


    def reward_scalation(self, rating):
        """
        Retorna la recompensa escalada en función de la calificación media de la película a recomendar
        """
        # la escala retorna 1 si el rating medio es 5, 0 si es 3 y -1 si es 1
        return (rating-3)/2


    def reset(self):
        """
        Reinicia el entorno, el estado actual se reinicia al estado inicial
        """
        self.state = self.initial_state


# Agente

In [4]:

class Agent:
    def __init__(self, env, gamma=0.9, alpha=0.1, epsilon=0.9, episodes=1000, qtable=None):

        # el agente tiene en memoria el ambiente
        self.environment = env
        # gamma factor de descuento
        self.gamma = gamma
        # alpha tasa de aprendizaje
        self.alpha = alpha
        # epsilon factor de exploración
        self.epsilon = epsilon
        # decay_rate tasa de decaimiento de epsilon
        self.decay_rate = 0.9

        # número de episodios
        self.episodes = episodes

        # Q-table se puede inicializar con valores ya entrenados del parámetro 'qtable'. 
        # Esto, junto a un valor de exploración pequeño, hará que el agente utilice esta política aprendida con antelación. 
        # Se inicializa como un diccionario vacío en caso de no recibir la política como parámetro, y 
        # los estados y acciones se irán añadiendo a medida que se vayan explorando (esto evita crear una tabla muy grande)
        self.qtable = qtable if qtable is not None else {}


    def run(self):
        """
        Entrenamiento del agente, se ejecutan los episodios hasta la condición de terminación de cada uno y se actualiza la Q table

        Returns
        -------
        dict
            Q table con los valores de los estados y acciones aprendidos
        """


        for episode in range(self.episodes):
            self.environment.reset()
            # estado actual (movie_id actual)
            state = self.environment.get_current_state()
            # done indica si el episodio ha terminado (si el agente ha llegado a 3 strikes)
            done = False

            while not done:
                # se escoge una acción (cambio de contenido) al azar o con base en la Q table determinado por epsilon
                action = self.random_action(state)

                reward, next_state, done = self.step(action)
                
                # si el estado no está en la Q table, se añade
                if state not in self.qtable:
                    self.qtable[state] = {action: 0}
                else:
                    # si la acción no está en la Q table, se añade
                    if action not in self.qtable[state]:
                        self.qtable[state][action] = 0

                # valor del estado actual en la Q table
                old_value = self.qtable[state][action]
                # valor del estado siguiente en la Q table
                next_max = max(self.qtable[next_state].values()) if next_state in self.qtable else 0
                # cálculo del nuevo valor del estado actual con base en la ecuación de Bellman
                new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)


                if action is not None:
                    self.qtable[state][action] = new_value
                else:
                    self.qtable[state][action] = reward
                    
                state = next_state
            # decaimiento de epsilon
            self.epsilon = max(self.epsilon * self.decay_rate,0.01)

        return self.qtable


    def random_action(self, current_state):
        """
        Retorna una acción al azar o con base en la Q table determinado por epsilon
        
        Parameters
        ----------
        current_state : int
            Estado actual (movie_id actual)

        Returns
        -------
        int
            Acción a realizar (movie_id a recomendar)
        """

        possible_actions = self.environment.get_possible_actions(current_state)

        # fase de exploración (adquirir conocimiento)
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(possible_actions)
        
        # fase de explotación del conocimiento
        else:
            # si aún no tiene conocimiento del estado actual, se escoge una al azar    
            if ~ (current_state in self.qtable.keys()):
                best_action = random.choice(possible_actions)

            # si ya tiene conocimiento, se escoge la acción con mayor valor
            else:
                max_value = max(self.qtable[current_state].values())
                max_keys = [key for key, value in self.qtable[current_state].items() if value == max_value]
                best_action = random.choice(max_keys)

            return best_action



    
    def step(self, action):       
        """
        Realiza una acción en el entorno, cambiar de contenido a recomendar
        
        Parameters
        ----------
        action : int
            Identificador de la película a recomendar (movie_id)
        
        Returns
        -------
        float
            Recompensa de realizar la acción"""

        return self.environment.do_action(action)
    

    def actions_values(self):
        """
        Retorna las películas recomendadas basándose en el valor más alto de todos los estados explorados
        
        Returns
        -------
        dict
            Diccionario con los estados y las acciones recomendadas
        dict
            Diccionario con los valores de los estados y acciones
        dict
            Diccionario con las películas recomendadas por cada película actual
        """

        actions = {}
        values = {}
        recommended = {}
        for state, action_values in self.qtable.items():
            
            # id de la película recomendada con mayor valor en la Q table por cada estado
            max_action = max(action_values, key=action_values.get)
            actions[state] = max_action
            
            values[state] = max(action_values.values())
            
            # nombre de película actual
            actual_movie = self.environment.get_movie_name(state)
            recommended[actual_movie] = self.environment.get_movie_name(max_action)
        
        return actions, values, recommended

# Funciones de persistencia de Q-Table

In [5]:
# Funciones para persistir y cargar los datos de la q-table

import json

def save_qtable(qtable, file_name):
    """
    Guarda la Q table en un archivo JSON

    Parameters
    ----------
    qtable : dict
        Q table con los valores de los estados y acciones aprendidos
    file_name : str
        Nombre del archivo donde se guardará la Q table
        
    """
    try:
        with open(file_name, 'w') as file:
            json.dump(qtable, file, indent=4)
        print(f"Qtable ha sido guardado en {file_name}")
    except IOError as e:
        print(f"An error occurred while saving data to {file_name}: {e}")




def load_qtable(file_name):
    """
    Carga la Q table desde un archivo JSON
    
    Parameters
    ----------
    file_name : str
        Nombre del archivo donde se encuentra la Q table
        
    Returns
    -------
    dict
        Q table con los valores de los estados y acciones aprendidos
    """
    try:
        with open(file_name, 'r') as file:
            data = json.load(file)
        return data
    except IOError as e:
        print(f"An error occurred while reading data from {file_name}: {e}")
        return None
    




# Datos de películas, contiene el id de la película, el año y el título
def read_movie_titles(csv_file_path):
    """
    Lee un archivo CSV con información de películas y retorna un DataFrame con los datos
    
    Parameters
    ----------
    csv_file_path : str
        Ruta del archivo CSV
    
    Returns
    -------
    pd.DataFrame
        DataFrame con los datos de las películas
        
    """

    list_of_movies = []

    # Open the CSV file for reading
    with open(csv_file_path, mode='r', newline='') as file:
        # Read the file line by line
        for line in file:
            # Strip any leading/trailing whitespace including newlines
            line = line.strip()
            
            # Split the line into exactly 3 parts: first three fields and the rest
            parts = line.split(',', 2)
            
            # Ensure that we have exactly 4 parts (the last part may contain commas)
            if len(parts) == 3:
                id_field = parts[0]
                name_field = parts[1]
                description_field = parts[2]
                
                list_of_movies.append((id_field, name_field, description_field))

            else:
                # Handle cases where the line does not contain enough commas
                print(f'Unexpected line format: {line}')

    movie_titles = pd.DataFrame(list_of_movies, columns=["movie_id", "year", "title"])

    return movie_titles

# Entrenamiento

In [7]:
# EJECUTAMOS EL AGENTE EN ESCENARIO DE ENTRENAMIENTO

env = Environment(data, read_movie_titles(r"Netflix_Prize_data\movie_titles.csv"))

#creación del agente
agent = Agent(env, gamma=0.9, alpha=0.1, epsilon=0.9, episodes=10) #episodios reducidos para efectos de demostración

# Ejecución del agente y obtenemos la Q-table con la política encontrada por el agente
qtable = agent.run()

# La política de esta Q-table la guardamos para poder usarla después
save_qtable(qtable, 'qtable.json')

actions, values, recommended = agent.actions_values()
recommended

Qtable ha sido guardado en qtable.json


{'Poirot: Lord Edgware Dies': 'Lilo and Stitch',
 'Lilo and Stitch': 'Aqua Teen Hunger Force: Vol. 3',
 'The Abyss': 'Buffy the Vampire Slayer: Season 6',
 'Buffy the Vampire Slayer: Season 6': 'The Three Stooges: Sing a Song of Six Pants',
 'The Three Stooges: Sing a Song of Six Pants': 'Lilo and Stitch',
 'Aqua Teen Hunger Force: Vol. 3': 'La Strada: Special Edition',
 'La Strada: Special Edition': 'Wallace & Gromit in Three Amazing Adventures',
 'Wallace & Gromit in Three Amazing Adventures': 'Arrested Development: Season 2',
 'Arrested Development: Season 2': "America's Next Top Model: Cycle 1",
 "America's Next Top Model: Cycle 1": 'Lord of the Rings: The Two Towers: Extended Edition',
 'Lord of the Rings: The Two Towers: Extended Edition': 'Strange Brew',
 'Strange Brew': 'The Bourne Identity',
 'The Bourne Identity': 'Spider-Man 2',
 'Spider-Man 2': 'Ace Ventura: Pet Detective',
 'Ace Ventura: Pet Detective': 'Life or Something Like It',
 'Life or Something Like It': 'Alias: Sea

In [8]:
# EJECUTAMOS EL AGENTE EN ESCENARIO DE USAR LA POLÍTICA APRENDIDA EN ENTRENAMIENTO

env = Environment(data, read_movie_titles(r"Netflix_Prize_data\movie_titles.csv"))

# Cargamos la política aprendida
qtable = load_qtable('qtable.json')

# creación del agente y pasamos la política como parámetro 'qtable'
# En esta ocasión vamos a utilizar un valor de exploración pequeño (epsilon),
# con el fin de que el agente explote la política encontrada lo más posible.
agent = Agent(env, gamma=0.9, alpha=0.1, epsilon=0.1, episodes=10, qtable=qtable) #episodios reducidos para efectos de demostración

#ejecución del agente
agent.run()
actions, values, recommended = agent.actions_values()
recommended

{'Poirot: Lord Edgware Dies': 'Lilo and Stitch',
 'Lilo and Stitch': 'Aqua Teen Hunger Force: Vol. 3',
 'The Abyss': 'Buffy the Vampire Slayer: Season 6',
 'Buffy the Vampire Slayer: Season 6': 'The Three Stooges: Sing a Song of Six Pants',
 'The Three Stooges: Sing a Song of Six Pants': 'Lilo and Stitch',
 'Aqua Teen Hunger Force: Vol. 3': 'La Strada: Special Edition',
 'La Strada: Special Edition': 'Wallace & Gromit in Three Amazing Adventures',
 'Wallace & Gromit in Three Amazing Adventures': 'Arrested Development: Season 2',
 'Arrested Development: Season 2': "America's Next Top Model: Cycle 1",
 "America's Next Top Model: Cycle 1": 'Lord of the Rings: The Two Towers: Extended Edition',
 'Lord of the Rings: The Two Towers: Extended Edition': 'Strange Brew',
 'Strange Brew': 'The Bourne Identity',
 'The Bourne Identity': 'Spider-Man 2',
 'Spider-Man 2': 'Ace Ventura: Pet Detective',
 'Ace Ventura: Pet Detective': 'Life or Something Like It',
 'Life or Something Like It': 'Alias: Sea